<a href="https://colab.research.google.com/github/gacemiamine77/gacemiamine77/blob/main/ClipRaster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.plot import show
from shapely.geometry import mapping
import matplotlib.pyplot as plt
import numpy as np

#------------beginning clipping function-------------------------------#
#This function clip raster Tif by a shapefile , it have three parameters
# RasterIn: the Geotiff to Clip
# shpClip the shapefile for clipping (.shp)
#RasterOut:  the clipped Geotiff as Result 

def clipRaster(RasterIn, ShpClip,RasterOut):
        df = gpd.read_file(ShpClip)
        #next line : if you want to filter a shapefile elsewere you comment it
        df = df.loc[df['id']!=0]
        print(df)
        inras =RasterIn 
        src  = rasterio.open(inras)
        def getFeatures(gdf):
            """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
            import json
            return [json.loads(gdf.to_json())['features'][0]['geometry']]
        coords = getFeatures(df)
        clipped_array, clipped_transform = mask(dataset=src, shapes=coords, crop=True)
        
        df = df.to_crs(src.crs)
        out_meta = src.meta.copy()
        out_meta.update({"driver": "GTiff",
                 "height": clipped_array.shape[1],
                 "width": clipped_array.shape[2],
                 "transform": clipped_transform})
        out_tif=RasterOut
        with rasterio.open(out_tif, "w", **out_meta) as dest:
            dest.write(clipped_array)
    
        clipped = rasterio.open(out_tif)
        #uncomment the next lines if you want to plotthe geotiff
        #fig, ax = plt.subplots(figsize=(8, 6))
        #p1 = df.plot(color=None,facecolor='none',edgecolor='red',linewidth = 2,ax=ax)
        #show(clipped, ax=ax)
        #ax.axis('off');


#------------end clipping function-------------------------------#

#------------Begining NDVI function-------------------------------#
# caclulate NDVi function with three parameters
#IR for the path of the InfraRed Band
#Red for the path of the Red Band
# NDVI_Out for the path of the ndvi result
def calcNDVI(IR,Red,NDVI_Out):
        f_IR=rasterio.open(IR) 
        f_Red=rasterio.open(Red) 
        B_IR=f_IR.read(1)
        B_Red=f_Red.read(1)
        
        
        #print(ImageSat_img.shape)
        nir=B_IR.astype('f4')
        red=B_Red.astype('f4')
        #np.seterr(divide='ignore', invalid='ignore')
        print(B_IR)
        ndvi = (nir - red) / (nir + red)
        #print(ndvi)
        # uncomment the next line if you want to plot the ndvi
        #plt.imshow(ndvi, cmap='terrain_r')
        out_meta = f_IR.meta.copy()
        with rasterio.open(NDVI_Out, "w", **out_meta) as dest:
            dest.write(ndvi,indexes=1)
#------------end ndvi function-------------------------------#                

#----------Main Program--------------------------------------------
from google.colab import drive
drive.mount('/content/drive')                
#clipRaster('c:/upwork/toufik/acidity_test1.tif','c:/upwork/toufik/limit_domaine.shp', "c:/upwork/toufik/clipped_exampless.tif")    

calcNDVI("/content/drive/MyDrive/upwork/IR.tif","/content/drive/MyDrive/upwork/Red.tif","/content/drive/MyDrive/upwork/NDVI_colab.tif")    

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
